<a href="https://colab.research.google.com/github/adelmuursepp/workshop-library/blob/main/Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Before you do anything...
1. Make a copy of this notebook so that any progress is saved to your copy. If you 
don't make a copy, your changes will be gone the next time you try to access this notebook.
2. Run the next cell to set up helper functions and examples. If you see strange errors of things not being defined, come back and run the set up cell again

In [ ]:
#@title Load libraries and data
from sklearn.datasets import make_moons, make_circles, make_classification
import pandas as pd
import itertools

import plotly.express as px
import plotly
import plotly.io as pio
import plotly.graph_objs as go
pio.templates.default = "simple_white"

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

def visualize_model(model, X, labels, K=200):
  x, y = X[:, 0], X[:, 1]
  L = x.min() - 0.1
  R = x.max() + 0.1
  U = y.max() + 0.1
  D = y.min() - 0.1

  df = pd.DataFrame()

  inputs = np.array( list( itertools.product(np.linspace(L,R, K), np.linspace(U,D, K))))
  probs = model.predict(inputs, verbose=0).flatten()
  pixels = np.zeros((K, K, 4))

  pixels[:,:,:3] = np.transpose(
          np.array(
            [
                plotly.colors.convert_to_RGB_255(p) for p in plotly.colors.sample_colorscale(
                    plotly.colors.get_colorscale("Picnic"), probs, 0, 1, "tuple"
                  )
            ]
          ).reshape(K, K, 3)
          ,[1,0,2] # Transpose
  )
  pixels[:,:,3] = 100 # Transparency

  fig = px.imshow(pixels)
  def x_to_px(x):
    return (x - L) * (K-1) / (R - L)
  
  def y_to_py(y):
    return (y-U) * (K-1) / (D - U)
  
  pixel_x = x_to_px(x)
  pixel_y = y_to_py(y)
  
  preds = model.predict(X).flatten()
  
  MARKER_SIZE=10
  fig.add_trace(
      go.Scatter(x=pixel_x[np.where(labels==0)], y=pixel_y[np.where(labels==0)], marker={"color":"blue", "size":MARKER_SIZE}, mode="markers", name="0")
  )
  
  fig.add_trace(
      go.Scatter(x=pixel_x[np.where(labels==1)], y=pixel_y[np.where(labels==1)], marker={"color":"red", "size":MARKER_SIZE}, mode="markers", name="1")
  )
  fig.add_trace(
      go.Scatter(x=pixel_x[labels != preds.round()], y=pixel_y[labels != preds.round()], marker={"color":"white", "symbol":"x", "size":MARKER_SIZE-3}, mode="markers", name="errors")
  )
  fig.show()

X, labels = make_moons(noise=0.3, random_state=0)
df = pd.DataFrame()

df["x"] = X[:,0]
df["y"] = X[:,1]
df["label"] = labels


In [ ]:
#@title Plot the data
px.scatter(df, x="x", y="y", color="label", height=700, width=700, color_continuous_scale="Picnic")

# Building and Training the Neural Network

The following implements a neural networks with two hidden layers. The first one has 16 neurons, and the second has 32. 

First, run the code as is to check that it works. Then, you can test your friends theory by removing the `activation = "relu"` optional arguments.

You can also try adding more layers or changing the size of the layers by adding more lines like `layers.Dense(SIZE),`.

Do not change the last layer!

Run the cell to train your neural network on the data. You can visualize what you learned in the next cell.

In [ ]:
model = keras.Sequential([
    layers.Dense(16, activation="relu", input_shape=[2]),
    layers.Dense(32, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=optimizer,
    metrics=['accuracy']
)

model.fit(X, labels, epochs=100)

# Let's see how we did!

Run the next cell to visualize the decision boundary of the model. The background color corresponds to the model's prediction at that location. The model learned well if the background colors match the placement of the datapoint (dots). The errors in the models predictions are marked by a white cross.

In [ ]:
visualize_model(model, X, labels)

Now, go back and test your friend's theory!

# Finding the equivalent single neuron neural network (optional)

`model.layers` is a list of layers and `layer.get_weights` returns the weights and biases of a single layer. For example, the following code iterates over the layers.

```
for layer in model236.layers:
  Wi, bi = layer.get_weights()
  ...
```

Create a model with no activation functions (except for the last layer, which still must be sigmoid), and then find the equivalent single output neuron.

You should compute the weights and bias of the output neuron and create a model like the following

```
model_equivalent = keras.Sequential([
    layers.Dense(1, activation="sigmoid", input_shape=[2])
])

model_equivalent.set_weights([COMPUTED_W, COMPUTED_b])
```

You can test if they are in fact equivalent by plotting their decision boundaries using `visualize_model`